In [2]:
import numpy as np
import open3d as o3d

### 스케일링

In [4]:
def scale_point_cloud_with_color(pcd, scale_factor):
    """
    Point cloud를 주어진 스케일 팩터만큼 확대 또는 축소하고 색상을 유지합니다.
    
    :param pcd: Open3D PointCloud 객체
    :param scale_factor: 스케일링 팩터 (1보다 크면 확대, 작으면 축소)
    :return: 스케일링된 Open3D PointCloud 객체
    """
    # 포인트 클라우드의 점들과 색상을 NumPy 배열로 가져옵니다
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    
    # 각 점의 좌표에 스케일 팩터를 곱합니다
    scaled_points = points * scale_factor
    
    # 새로운 Open3D PointCloud 객체를 생성합니다
    scaled_pcd = o3d.geometry.PointCloud()
    
    # 스케일링된 점들과 원래의 색상을 새 PointCloud 객체에 설정합니다
    scaled_pcd.points = o3d.utility.Vector3dVector(scaled_points)
    scaled_pcd.colors = o3d.utility.Vector3dVector(colors)
    
    return scaled_pcd

pcd = o3d.io.read_point_cloud("./data/shoe.ply")
scaled_pcd = scale_point_cloud_with_color(pcd, 1.5)  # 1.5배 확대
o3d.visualization.draw([scaled_pcd])

### 샘플링

In [9]:
def random_sampling_with_color(pcd, num_samples):
    """
    Point cloud에서 무작위로 지정된 개수의 점들을 샘플링하고 색상을 유지합니다.
    
    :param pcd: Open3D PointCloud 객체
    :param num_samples: 샘플링할 점의 개수
    :return: 샘플링된 Open3D PointCloud 객체
    """
    # 포인트 클라우드의 점들과 색상을 NumPy 배열로 가져옵니다
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    
    # 전체 점의 개수를 구합니다
    num_points = points.shape[0]
    
    # 무작위로 인덱스를 선택합니다
    sampled_indices = np.random.choice(num_points, num_samples, replace=False)
    
    # 선택된 인덱스에 해당하는 점들과 색상만 추출합니다
    sampled_points = points[sampled_indices]
    sampled_colors = colors[sampled_indices]
    
    # 새로운 Open3D PointCloud 객체를 생성합니다
    sampled_pcd = o3d.geometry.PointCloud()
    
    # 샘플링된 점들과 색상을 새 PointCloud 객체에 설정합니다
    sampled_pcd.points = o3d.utility.Vector3dVector(sampled_points)
    sampled_pcd.colors = o3d.utility.Vector3dVector(sampled_colors)
    
    return sampled_pcd

pcd = o3d.io.read_point_cloud("./data/shoe.ply")
sampled_pcd = random_sampling_with_color(pcd, 1000)  # 1000개의 점을 무작위로 샘플링
o3d.visualization.draw([sampled_pcd])

### 변형

In [8]:
def sine_wave_deformation_with_color(pcd, amplitude, frequency):
    """
    Point cloud에 사인파 변형을 적용하고 색상을 유지합니다.
    
    :param pcd: Open3D PointCloud 객체
    :param amplitude: 사인파의 진폭
    :param frequency: 사인파의 주파수
    :return: 변형된 Open3D PointCloud 객체
    """
    # 포인트 클라우드의 점들과 색상을 NumPy 배열로 가져옵니다
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    
    # x 좌표를 기준으로 y 좌표에 사인파 변형을 적용합니다
    points[:, 1] += amplitude * np.sin(frequency * points[:, 0])
    
    # 새로운 Open3D PointCloud 객체를 생성합니다
    deformed_pcd = o3d.geometry.PointCloud()
    
    # 변형된 점들과 원래의 색상을 새 PointCloud 객체에 설정합니다
    deformed_pcd.points = o3d.utility.Vector3dVector(points)
    deformed_pcd.colors = o3d.utility.Vector3dVector(colors)
    
    return deformed_pcd

pcd = o3d.io.read_point_cloud("./data/shoe.ply")
deformed_pcd = sine_wave_deformation_with_color(pcd, amplitude=0.1, frequency=5)
o3d.visualization.draw([deformed_pcd])

### 세그멘테이션

In [4]:
def plane_segmentation_with_color(pcd, distance_threshold, ransac_n, num_iterations):
    """
    RANSAC 알고리즘을 사용하여 point cloud에서 평면을 분리하고 색상을 유지합니다.
    
    :param pcd: Open3D PointCloud 객체
    :param distance_threshold: 점과 평면 사이의 최대 거리
    :param ransac_n: RANSAC에 사용할 점의 개수
    :param num_iterations: RANSAC 반복 횟수
    :return: (평면 PointCloud, 나머지 PointCloud)
    """
    # RANSAC을 사용하여 평면 세그멘테이션을 수행합니다
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,
                                             ransac_n=ransac_n,
                                             num_iterations=num_iterations)
    
    # 평면과 나머지 부분을 분리합니다 (색상 정보가 자동으로 유지됩니다)
    plane_cloud = pcd.select_by_index(inliers)
    rest_cloud = pcd.select_by_index(inliers, invert=True)
    
    return plane_cloud, rest_cloud

# 사용 예:
pcd = o3d.io.read_point_cloud("./data/shoe.ply")
plane_cloud, rest_cloud = plane_segmentation_with_color(pcd, distance_threshold=0.01, ransac_n=3, num_iterations=1000)
o3d.visualization.draw([rest_cloud])